# A simple MNIST example with keras

This tutorial at first assumes you already have some net that is good for the task and you only want to feed it your data and train it.

There are further examples in the examples folder as python code.
Just dig arround a bit.
The examples should be nicely documented.

## Load the data

For loading the data we simply include the mnist classification dataset loader.
The data should be stored in the data/mnist folder, to keep stuff clean.
We can select if the training or the test data should be loaded.
In this case we want to load both.
We need also data to validate our model.

The prepare_data method is a method that converts the input images/data into the format the network needs.
For now we will simply use the mnist cnn prepare_data method provided by the framework.

In [ ]:
from datasets.classification.mnist import mnist
from keras_models.mnist_cnn import prepare_data

images, labels = mnist("data/mnist", "train", prepare_data)
test_images, test_labels = mnist("data/mnist", "test", prepare_data)

print("Training Images: %d" % len(images))
print("Test Images: %d" % len(test_images))

## Creating the model

Next stop we will create an mnist model.

We will use a predefined model for mnist for now.
Since the framework provides one.

We want to plot the model.

In [ ]:
from keras_models.mnist_cnn import mnist_toy_net
from keras.utils import plot_model

print("\nCreating Model: mnist_toy_net")
model = mnist_toy_net()
plot_model(model, show_shapes=True)


## Adding an optimizer and loss

In keras adding a loss and an optimizer is done with the two lines below.
Create the optimizer and then compile the model with a loss and an optimizer

In [ ]:
from keras.optimizers import SGD

print("\nCreate SGD Optimizer")
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])

## Training

To train the model simply fit it to the data.

In [ ]:
print("\nFit model...")
model.fit(x=images, y=labels, batch_size=128, epochs=200, validation_data=(test_images, test_labels))

## Save the model for deploying

After training it is important to save the model definition and weights in a file for later usage.
A pro tipp is to save the model using a timestamp, so you do not overwrite it when testing out multiple configurations.

In [ ]:
import time
import datetime
time_str = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H.%M.%S')

model_json = model.to_json()
with open("models/weights/mnist_toy_net_%s.json" % time_str, "w") as json_file:
    json_file.write(model_json)
model.save_weights("models/weights/mnist_toy_net_%s.h5" % time_str)

## Load a model for prediction

A model can be simply loaded for prediction by providing a weights_path to the constructor.
You can then predict your images by using the model.predict function.
After compiling the model with a loss again.

In [ ]:
import numpy as np

# load trained model
model = mnist_toy_net(weights_path="models/weights/mnist_toy_net_2018-01-09_20.04.47.h5")

# Compile model
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])

# Predict
classes = np.argmax(model.predict(test_images), axis=1)
print(classes)

## Hungry for more?

Have a look at the mnist.py example, which is largely similar to this tutorial but adds a hook for loss plots to the training process.

There is also the tinypersonnet.py which trains a network on a dataset saved in named folders. The tinypersonnet_eval.py evaluates/deploys that network.

Also currently under development is the lfw.py example for labeled faces in the wild.
Basically there are examples for every implemented dataset that can be read.